In [30]:
!pip install mercantile
!pip install mapbox_vector_tile
!pip install vt2geojson
!pip install osmnx

In [31]:
import os
from google.colab import drive
import osmnx as ox
import networkx as nx
import pickle
import json
import time
import requests
import mercantile, mapbox_vector_tile
from vt2geojson.tools import vt_bytes_to_geojson


# Mount to Gdrive


In [32]:
# connect to drive
drive.mount('/gdrive')

path = '/gdrive/MyDrive/berlin_bike_CV/'
folder = 'metadata'
mapillary_folder = 'mapillary'

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Download and Save Graph of Berlin

In [43]:
# define Area and main name to save

# graph of Berlin
# place_name = "Berlin, Germany"
# G = ox.graph_from_place(place_name, network_type="bike")
# point: [lat, lon]
# place_dsr = [52.500632881796925, 13.334863741376868]
# G = ox.graph_from_point(place_dsr, dist=1000, network_type="bike")
# [north, south, east, west]
bb_dsr = [52.534947724864615, 52.48412181578994, 13.379150390625, 13.295654296875]
G = ox.graph_from_bbox(*bb_dsr, network_type='bike')

name = 'DSR'

In [44]:

# Find bounding box
gdf = ox.graph_to_gdfs(G, nodes=False, edges=True)
bounding_box = gdf.total_bounds

# # Anzeigen der Koordinaten der umgebenden Box
print("Umschließende Box:")
print("Minimale Breite (westlichster Punkt):", bounding_box[0])
print("Minimale Länge (südlichster Punkt):", bounding_box[1])
print("Maximale Breite (östlichster Punkt):", bounding_box[2])
print("Maximale Länge (nördlichster Punkt):", bounding_box[3])


Umschließende Box:
Minimale Breite (westlichster Punkt): 13.2956688
Minimale Länge (südlichster Punkt): 52.4839151
Maximale Breite (östlichster Punkt): 13.3791321
Maximale Länge (nördlichster Punkt): 52.5349833


In [64]:
# save original graph
graph_name = '_graph_origin.pkl'
graph_path = os.path.join(path, folder, name + graph_name)
with open(graph_path, 'wb') as pickle_file:
    pickle.dump(G, pickle_file)



In [65]:
# save graph for weighting
graph_name = '_graph_weights.pkl'
graph_path = os.path.join(path, folder, name + graph_name)
with open(graph_path, 'wb') as pickle_file:
    pickle.dump(G, pickle_file)

In [1]:
# with open(graph_path, 'rb') as pickle_file:
#     G_reconstructed = pickle.load(pickle_file)


# ox.plot_graph(G)

NameError: ignored

# Find Set of Tiles for Berlin

In [48]:

# define an empty geojson as output
# output= { "type": "FeatureCollection", "features": [] }

# vector tile endpoints -- change this in the API request to reference the correct endpoint
tile_coverage = 'mly1_public'

# tile layer depends which vector tile endpoints:
# 1. if map features or traffic signs, it will be "point" always
# 2. if looking for coverage, it will be "image" for points, "sequence" for lines, or "overview" for far zoom
tile_layer = "image"

### TODO: save token in gdrive !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Mapillary access token -- user should provide their own
access_token = 'MLY|6215450018543661|0bb93dcff4f69ed8b1a83e8474f34a89'

# a bounding box in [east_lng,_south_lat,west_lng,north_lat] format
# bounding_box = [13.0918841 52.3404293 13.7556118 52.6731193]
west, south, east, north = bounding_box

# get the list of tiles with x and y coordinates which intersect our bounding box
# MUST be at zoom level 14 where the data is available, other zooms currently not supported
tiles = list(mercantile.tiles(west, south, east, north, 14))

In [49]:
# print(len(tiles))
# print(tiles[0])
# val = f"{str(tiles[0].x)}_{str(tiles[0].y)}_{str(tiles[0].z)}.geojson"
# quadkey = mercantile.quadkey(tiles[0])
# print(quadkey)
# quadkey = mercantile.quadkey(tiles[-1])
# print(quadkey)
# tile = mercantile.quadkey_to_tile(quadkey)
# print(tile)


In [50]:
# print(mercantile.bounds(tiles[0]))
# print(mercantile.bounds(tiles[1]))
# print(mercantile.bounds(tiles[2]))
# print(mercantile.bounds(tiles[3]))
# print(mercantile.bounds(tiles[0])[2] - 1.9* (mercantile.bounds(tiles[0])[2]-mercantile.bounds(tiles[0])[0]))
# print(mercantile.bounds(tiles[0])[2] + 1.9* (mercantile.bounds(tiles[0])[2]-mercantile.bounds(tiles[0])[0]))
# print(mercantile.bounds(tiles[0])[3] - 1.9* (mercantile.bounds(tiles[0])[3]-mercantile.bounds(tiles[0])[1]))
# print(mercantile.bounds(tiles[0])[3] + 1.9* (mercantile.bounds(tiles[0])[3]-mercantile.bounds(tiles[0])[1]))

# Divide and Save Set of Edges of Graph per Tile

In [106]:
def create_edges():
  edges = []

  for u, v, k in G.edges(keys=True):
          edges.append((u, v, k))
  return edges

In [107]:
def extract_edges(tile, remaining_edges):

    edges = {}
    remove_edges = []
    min_lon, min_lat, max_lon, max_lat = mercantile.bounds(tile)

    for (u, v, k) in remaining_edges:
        u_x, u_y = G.nodes[u]['x'], G.nodes[u]['y']
        v_x, v_y = G.nodes[v]['x'], G.nodes[v]['y']

        if (min_lon <= u_x <= max_lon and min_lat <= u_y <= max_lat) or (min_lon <= v_x <= max_lon and min_lat <= v_y <= max_lat):
          edges[(u, v, k)] = []
          remove_edges.append((u, v, k))

    for edge in reversed(remove_edges):
        remaining_edges.remove(edge)

    return edges, remaining_edges

In [108]:
def joining_edges_to_tiles(tiles):
    remaining_edges = create_edges()
    # print(len(remaining_edges))
    tiles_data = {}

    for i, tile in enumerate(tiles):
        edges , remaining_edges = extract_edges(tile, remaining_edges)
        if edges:
          tiles_data[mercantile.quadkey(tile)] = edges

    return tiles_data

In [109]:
# TODO devide edgeset per tile
tiles_data = joining_edges_to_tiles(tiles)


In [110]:
# save tiles with dict of edges in gdrive
file_name = '_tiles_edges.pkl'
file_path = os.path.join(path, folder, name + file_name)
with open(file_path, 'wb') as pickle_file:
    pickle.dump(tiles_data, pickle_file)

In [111]:
with open(file_path, 'rb') as pickle_file:
  tiles_reconstructed = pickle.load(pickle_file)

In [112]:
print(len(tiles_reconstructed))
for key, value in tiles_reconstructed.items():
    print(key)
    print(value)

16
12021023233301
{(26662368, 26682529, 0): [], (26662368, 9007281304, 0): [], (26662368, 26784424, 0): [], (26662377, 1516808200, 0): [], (26662377, 26662449, 0): [], (26662389, 1516808200, 0): [], (26662389, 5503222927, 0): [], (26662389, 26736904, 0): [], (26662394, 27306842, 0): [], (26662394, 26662411, 0): [], (26662394, 26662458, 0): [], (26662394, 26736904, 0): [], (26662411, 26784426, 0): [], (26662411, 1516808167, 0): [], (26662411, 26662394, 0): [], (26662413, 26736904, 0): [], (26662413, 7100773720, 0): [], (26662413, 1594609564, 0): [], (26662413, 26750501, 0): [], (26662420, 30423984, 0): [], (26662420, 26662429, 0): [], (26662420, 26757492, 0): [], (26662420, 26758734, 0): [], (26662429, 26662420, 0): [], (26662429, 26736800, 0): [], (26662429, 6724108544, 0): [], (26662429, 3305073638, 0): [], (26662449, 26750500, 0): [], (26662449, 198960345, 0): [], (26662449, 7100773720, 0): [], (26662449, 26662377, 0): [], (26662458, 26750492, 0): [], (26662458, 26662394, 0): [], (26

# Download and Save Mapillary Metadata per Tile

In [76]:
# TODO take Mapillary API downloading rates into acount!!!
sleep_rate = 0.5

# loop through list of tiles to get tile z/x/y to plug in to Mapillary endpoints and make request
for tile_quadkey in tiles_data:
    time.sleep(sleep_rate)
    tile = mercantile.quadkey_to_tile(tile_quadkey)
    tile_url = 'https://tiles.mapillary.com/maps/vtp/{}/2/{}/{}/{}?access_token={}'.format(tile_coverage,tile.z,tile.x,tile.y,access_token)
    response = requests.get(tile_url)
    data = vt_bytes_to_geojson(response.content, tile.x, tile.y, tile.z,layer=tile_layer)

    # TODO: put {tile: data} in berlin_mapillary_image_data.json
    tile_data = { "type": "FeatureCollection", "features": [] }

    # push to output geojson object if yes
    for feature in data['features']:

        # get lng,lat of each feature
        lng = feature['geometry']['coordinates'][0]
        lat = feature['geometry']['coordinates'][1]

        # ensure feature falls inside bounding box since tiles can extend beyond
        if lng > west and lng < east and lat > south and lat < north:

            if feature["properties"]["is_pano"]:
                continue

            tile_data['features'].append(feature)

    # save data per tile
    file_name = tile_quadkey + '.geojson'
    file_path = os.path.join(path, folder, mapillary_folder, file_name)
    with open(file_path, 'w') as f:
        json.dump(tile_data, f)




In [77]:


print('Done')

Done


In [71]:
# with open(file_path, 'r') as f:
#   mapillary_reconstructed = json.load(f)

In [1]:
# print(len(mapillary_reconstructed))
# for key, value in mapillary_reconstructed.items():
#     print(key)
#     print(value)
#     time.sleep(1)


NameError: ignored